In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [2]:
import numpy as np
import tflearn
import tensorflow as tf
import random

In [3]:
# import our chat-bot entities file
import json
with open('entities.json') as json_data:
    entities = json.load(json_data)

In [4]:
words = [] #Design the Vocabulary (unique words)
classes = []
documents = []
# loop through each sentence in our intents patterns
for entity in entities['entities']:
    for example in entity['examples']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(example)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, entity['tag']))
        # add to our classes list
        if entity['tag'] not in classes:
            classes.append(entity['tag']) 

In [5]:
# stem and lower each word and remove duplicates
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words ]
words = sorted(list(set(words)))

In [6]:
words

['connect', 'cor', 'devop', 'easyrun', 'par', 'tal', 'tun']

In [7]:
# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


7 documents
2 classes ['accretio-groups', 'location']
7 unique stemmed words ['connect', 'cor', 'devop', 'easyrun', 'par', 'tal', 'tun']


In [8]:
# create our training data
training = []
output = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the examples 
    example_words = doc[0] 
    # stem each word
    example_words = [stemmer.stem(word.lower()) for word in example_words]
    # create our bag of words array
    # mark the presence of words as a boolean value, 0 for absent, 1 for present.
    for w in words:
        bag.append(1) if w in example_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [9]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [10]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_entities_logs', tensorboard_verbose=3)
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model_entities.tflearn')
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data_entities", "wb" ) )

Instructions for updating:
keep_dims is deprecated, use keepdims instead
---------------------------------
Run id: XFKSM1
Log directory: tflearn_entities_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 7
Validation samples: 0
--
Training Step: 1  | time: 0.093s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 7/7
--
Training Step: 2  | total loss: 0.62383 | time: 0.002s
| Adam | epoch: 002 | loss: 0.62383 - acc: 0.3857 -- iter: 7/7
--
Training Step: 3  | total loss: 0.68015 | time: 0.003s
| Adam | epoch: 003 | loss: 0.68015 - acc: 0.6545 -- iter: 7/7
--
Training Step: 4  | total loss: 0.69093 | time: 0.003s
| Adam | epoch: 004 | loss: 0.69093 - acc: 0.6994 -- iter: 7/7
--
Training Step: 5  | total loss: 0.69111 | time: 0.003s
| Adam | epoch: 005 | loss: 0.69111 - acc: 0.7097 -- iter: 7/7
--
Training Step: 6  | total loss: 0.69087 | time: 0.003s
| Adam | epoch: 006 | loss: 0.69087

Training Step: 57  | total loss: 0.63715 | time: 0.003s
| Adam | epoch: 057 | loss: 0.63715 - acc: 0.7143 -- iter: 7/7
--
Training Step: 58  | total loss: 0.63715 | time: 0.003s
| Adam | epoch: 058 | loss: 0.63715 - acc: 0.7143 -- iter: 7/7
--
Training Step: 59  | total loss: 0.63490 | time: 0.003s
| Adam | epoch: 059 | loss: 0.63490 - acc: 0.7143 -- iter: 7/7
--
Training Step: 60  | total loss: 0.63259 | time: 0.003s
| Adam | epoch: 060 | loss: 0.63259 - acc: 0.7143 -- iter: 7/7
--
Training Step: 61  | total loss: 0.63023 | time: 0.002s
| Adam | epoch: 061 | loss: 0.63023 - acc: 0.7143 -- iter: 7/7
--
Training Step: 62  | total loss: 0.63025 | time: 0.002s
| Adam | epoch: 062 | loss: 0.63025 - acc: 0.7143 -- iter: 7/7
--
Training Step: 63  | total loss: 0.62747 | time: 0.002s
| Adam | epoch: 063 | loss: 0.62747 - acc: 0.7143 -- iter: 7/7
--
Training Step: 64  | total loss: 0.62469 | time: 0.003s
| Adam | epoch: 064 | loss: 0.62469 - acc: 0.7143 -- iter: 7/7
--
Training Step: 65  | tot

Training Step: 116  | total loss: 0.44103 | time: 0.002s
| Adam | epoch: 116 | loss: 0.44103 - acc: 0.7143 -- iter: 7/7
--
Training Step: 117  | total loss: 0.43622 | time: 0.002s
| Adam | epoch: 117 | loss: 0.43622 - acc: 0.7143 -- iter: 7/7
--
Training Step: 118  | total loss: 0.43144 | time: 0.002s
| Adam | epoch: 118 | loss: 0.43144 - acc: 0.7143 -- iter: 7/7
--
Training Step: 119  | total loss: 0.42668 | time: 0.002s
| Adam | epoch: 119 | loss: 0.42668 - acc: 0.7143 -- iter: 7/7
--
Training Step: 120  | total loss: 0.42193 | time: 0.002s
| Adam | epoch: 120 | loss: 0.42193 - acc: 0.7143 -- iter: 7/7
--
Training Step: 121  | total loss: 0.41719 | time: 0.003s
| Adam | epoch: 121 | loss: 0.41719 - acc: 0.7143 -- iter: 7/7
--
Training Step: 122  | total loss: 0.41245 | time: 0.003s
| Adam | epoch: 122 | loss: 0.41245 - acc: 0.7143 -- iter: 7/7
--
Training Step: 123  | total loss: 0.40770 | time: 0.003s
| Adam | epoch: 123 | loss: 0.40770 - acc: 0.7143 -- iter: 7/7
--
Training Step: 1

Training Step: 175  | total loss: 0.32485 | time: 0.006s
| Adam | epoch: 175 | loss: 0.32485 - acc: 0.9026 -- iter: 7/7
--
Training Step: 176  | total loss: 0.30687 | time: 0.002s
| Adam | epoch: 176 | loss: 0.30687 - acc: 0.9124 -- iter: 7/7
--
Training Step: 177  | total loss: 0.29055 | time: 0.002s
| Adam | epoch: 177 | loss: 0.29055 - acc: 0.9211 -- iter: 7/7
--
Training Step: 178  | total loss: 0.27569 | time: 0.002s
| Adam | epoch: 178 | loss: 0.27569 - acc: 0.9290 -- iter: 7/7
--
Training Step: 179  | total loss: 0.26215 | time: 0.004s
| Adam | epoch: 179 | loss: 0.26215 - acc: 0.9361 -- iter: 7/7
--
Training Step: 180  | total loss: 0.23844 | time: 0.003s
| Adam | epoch: 180 | loss: 0.23844 - acc: 0.9425 -- iter: 7/7
--
Training Step: 181  | total loss: 0.23844 | time: 0.002s
| Adam | epoch: 181 | loss: 0.23844 - acc: 0.9483 -- iter: 7/7
--
Training Step: 182  | total loss: 0.22803 | time: 0.003s
| Adam | epoch: 182 | loss: 0.22803 - acc: 0.9534 -- iter: 7/7
--
Training Step: 1

Training Step: 234  | total loss: 0.18776 | time: 0.003s
| Adam | epoch: 234 | loss: 0.18776 - acc: 0.9555 -- iter: 7/7
--
Training Step: 235  | total loss: 0.17412 | time: 0.003s
| Adam | epoch: 235 | loss: 0.17412 - acc: 0.9599 -- iter: 7/7
--
Training Step: 236  | total loss: 0.16182 | time: 0.003s
| Adam | epoch: 236 | loss: 0.16182 - acc: 0.9639 -- iter: 7/7
--
Training Step: 237  | total loss: 0.15072 | time: 0.003s
| Adam | epoch: 237 | loss: 0.15072 - acc: 0.9675 -- iter: 7/7
--
Training Step: 238  | total loss: 0.14069 | time: 0.003s
| Adam | epoch: 238 | loss: 0.14069 - acc: 0.9708 -- iter: 7/7
--
Training Step: 239  | total loss: 0.13163 | time: 0.003s
| Adam | epoch: 239 | loss: 0.13163 - acc: 0.9763 -- iter: 7/7
--
Training Step: 240  | total loss: 0.12343 | time: 0.002s
| Adam | epoch: 240 | loss: 0.12343 - acc: 0.9763 -- iter: 7/7
--
Training Step: 241  | total loss: 0.11600 | time: 0.002s
| Adam | epoch: 241 | loss: 0.11600 - acc: 0.9787 -- iter: 7/7
--
Training Step: 2

...

Training Step: 765  | total loss: 0.28305 | time: 0.003s
| Adam | epoch: 765 | loss: 0.28305 - acc: 0.9219 -- iter: 7/7
--
Training Step: 766  | total loss: 0.25908 | time: 0.003s
| Adam | epoch: 766 | loss: 0.25908 - acc: 0.9297 -- iter: 7/7
--
Training Step: 767  | total loss: 0.23753 | time: 0.003s
| Adam | epoch: 767 | loss: 0.23753 - acc: 0.9368 -- iter: 7/7
--
Training Step: 768  | total loss: 0.20074 | time: 0.003s
| Adam | epoch: 768 | loss: 0.20074 - acc: 0.9431 -- iter: 7/7
--
Training Step: 769  | total loss: 0.20074 | time: 0.003s
| Adam | epoch: 769 | loss: 0.20074 - acc: 0.9488 -- iter: 7/7
--
Training Step: 770  | total loss: 0.18507 | time: 0.005s
| Adam | epoch: 770 | loss: 0.18507 - acc: 0.9539 -- iter: 7/7
--
Training Step: 771  | total loss: 0.15829 | time: 0.003s
| Adam | epoch: 771 | loss: 0.15829 - acc: 0.9585 -- iter: 7/7
--
Training Step: 772  | total loss: 0.14686 | time: 0.003s
| Adam | epoch: 772 | loss: 0.14686 - acc: 0.9627 -- iter: 7/7
--
Training Step: 7

Training Step: 824  | total loss: 0.19670 | time: 0.003s
| Adam | epoch: 824 | loss: 0.19670 - acc: 0.9437 -- iter: 7/7
--
Training Step: 825  | total loss: 0.18071 | time: 0.005s
| Adam | epoch: 825 | loss: 0.18071 - acc: 0.9493 -- iter: 7/7
--
Training Step: 826  | total loss: 0.16632 | time: 0.004s
| Adam | epoch: 826 | loss: 0.16632 - acc: 0.9544 -- iter: 7/7
--
Training Step: 827  | total loss: 0.15338 | time: 0.003s
| Adam | epoch: 827 | loss: 0.15338 - acc: 0.9590 -- iter: 7/7
--
Training Step: 828  | total loss: 0.14174 | time: 0.003s
| Adam | epoch: 828 | loss: 0.14174 - acc: 0.9631 -- iter: 7/7
--
Training Step: 829  | total loss: 0.14174 | time: 0.002s
| Adam | epoch: 829 | loss: 0.14174 - acc: 0.9668 -- iter: 7/7
--
Training Step: 830  | total loss: 0.31149 | time: 0.002s
| Adam | epoch: 830 | loss: 0.31149 - acc: 0.9216 -- iter: 7/7
--
Training Step: 831  | total loss: 0.28408 | time: 0.002s
| Adam | epoch: 831 | loss: 0.28408 - acc: 0.9216 -- iter: 7/7
--
Training Step: 8

Training Step: 883  | total loss: 0.29671 | time: 0.002s
| Adam | epoch: 883 | loss: 0.29671 - acc: 0.9174 -- iter: 7/7
--
Training Step: 884  | total loss: 0.24913 | time: 0.005s
| Adam | epoch: 884 | loss: 0.24913 - acc: 0.9256 -- iter: 7/7
--
Training Step: 885  | total loss: 0.24913 | time: 0.006s
| Adam | epoch: 885 | loss: 0.24913 - acc: 0.9331 -- iter: 7/7
--
Training Step: 886  | total loss: 0.39629 | time: 0.002s
| Adam | epoch: 886 | loss: 0.39629 - acc: 0.8826 -- iter: 7/7
--
Training Step: 887  | total loss: 0.36140 | time: 0.002s
| Adam | epoch: 887 | loss: 0.36140 - acc: 0.8944 -- iter: 7/7
--
Training Step: 888  | total loss: 0.33007 | time: 0.002s
| Adam | epoch: 888 | loss: 0.33007 - acc: 0.9049 -- iter: 7/7
--
Training Step: 889  | total loss: 0.30192 | time: 0.003s
| Adam | epoch: 889 | loss: 0.30192 - acc: 0.9144 -- iter: 7/7
--
Training Step: 890  | total loss: 0.27663 | time: 0.003s
| Adam | epoch: 890 | loss: 0.27663 - acc: 0.9230 -- iter: 7/7
--
Training Step: 8

Training Step: 942  | total loss: 0.37506 | time: 0.002s
| Adam | epoch: 942 | loss: 0.37506 - acc: 0.8861 -- iter: 7/7
--
Training Step: 943  | total loss: 0.34302 | time: 0.003s
| Adam | epoch: 943 | loss: 0.34302 - acc: 0.8975 -- iter: 7/7
--
Training Step: 944  | total loss: 0.31423 | time: 0.002s
| Adam | epoch: 944 | loss: 0.31423 - acc: 0.9077 -- iter: 7/7
--
Training Step: 945  | total loss: 0.28837 | time: 0.002s
| Adam | epoch: 945 | loss: 0.28837 - acc: 0.9170 -- iter: 7/7
--
Training Step: 946  | total loss: 0.26513 | time: 0.002s
| Adam | epoch: 946 | loss: 0.26513 - acc: 0.9253 -- iter: 7/7
--
Training Step: 947  | total loss: 0.24423 | time: 0.003s
| Adam | epoch: 947 | loss: 0.24423 - acc: 0.9327 -- iter: 7/7
--
Training Step: 948  | total loss: 0.22545 | time: 0.003s
| Adam | epoch: 948 | loss: 0.22545 - acc: 0.9395 -- iter: 7/7
--
Training Step: 949  | total loss: 0.20855 | time: 0.003s
| Adam | epoch: 949 | loss: 0.20855 - acc: 0.9455 -- iter: 7/7
--
Training Step: 9

INFO:tensorflow:/home/helmi/Jupyter/Chatbot1/model_entities.tflearn is not in all_model_checkpoint_paths. Manually adding it.
